# Unsupervised Learning

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/git/https%3A%2F%2Fgitlab.in2p3.fr%2Fenergy4climate%2Fpublic%2Feducation%2Fmachine_learning_for_climate_and_energy/master?filepath=book%2Fnotebooks%2F2_unsupervised_learning.ipynb)

<div class="alert alert-block alert-warning">
    <b>Prerequisites</b>
    
- 
</div>

<div class="alert alert-block alert-info">
    <b>Learning Outcomes</b>
    
- 
</div>